In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as opt
from torch.utils.data import DataLoader,TensorDataset
from torch.nn.utils.rnn import pad_packed_sequence,pack_padded_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import scipy.io as sio
import pickle
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
blosum62 = {
        'A': [4, -1, -2, -2, 0,  -1, -1, 0, -2,  -1, -1, -1, -1, -2, -1, 1,  0,  -3, -2, 0],  # A
        'R': [-1, 5,  0, -2, -3, 1,  0,  -2, 0,  -3, -2, 2,  -1, -3, -2, -1, -1, -3, -2, -3], # R
        'N': [-2, 0,  6,  1,  -3, 0,  0,  0,  1,  -3, -3, 0,  -2, -3, -2, 1,  0,  -4, -2, -3], # N
        'D': [-2, -2, 1,  6,  -3, 0,  2,  -1, -1, -3, -4, -1, -3, -3, -1, 0,  -1, -4, -3, -3], # D
        'C': [0,  -3, -3, -3, 9,  -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1], # C
        'Q': [-1, 1,  0,  0,  -3, 5,  2,  -2, 0,  -3, -2, 1,  0,  -3, -1, 0,  -1, -2, -1, -2], # Q
        'E': [-1, 0,  0,  2,  -4, 2,  5,  -2, 0,  -3, -3, 1,  -2, -3, -1, 0,  -1, -3, -2, -2], # E
        'G': [0,  -2, 0,  -1, -3, -2, -2, 6,  -2, -4, -4, -2, -3, -3, -2, 0,  -2, -2, -3, -3], # G
        'H': [-2, 0,  1,  -1, -3, 0,  0,  -2, 8,  -3, -3, -1, -2, -1, -2, -1, -2, -2, 2,  -3], # H
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4,  2,  -3, 1,  0,  -3, -2, -1, -3, -1, 3],  # I
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2,  4,  -2, 2,  0,  -3, -2, -1, -2, -1, 1],  # L
        'K': [-1, 2,  0,  -1, -3, 1,  1,  -2, -1, -3, -2, 5,  -1, -3, -1, 0,  -1, -3, -2, -2], # K
        'M': [-1, -1, -2, -3, -1, 0,  -2, -3, -2, 1,  2,  -1, 5,  0,  -2, -1, -1, -1, -1, 1],  # M
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0,  0,  -3, 0,  6,  -4, -2, -2, 1,  3,  -1], # F
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7,  -1, -1, -4, -3, -2], # P
        'S': [1,  -1, 1,  0,  -1, 0,  0,  0,  -1, -2, -2, 0,  -1, -2, -1, 4,  1,  -3, -2, -2], # S
        'T': [0,  -1, 0,  -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1,  5,  -2, -2, 0],  # T
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1,  -4, -3, -2, 11, 2,  -3], # W
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2,  -1, -1, -2, -1, 3,  -3, -2, -2, 2,  7,  -1], # Y
        'V': [0,  -3, -3, -3, -1, -2, -2, -3, -3, 3,  1,  -2, 1,  -1, -2, -2, 0,  -3, -1, 4],  # V
        '-': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # -
    }

def normalization(dataset):
    minimum = dataset.min(axis=0)
    maximum = dataset.max(axis=0)
    dataset = np.divide((dataset - minimum), (maximum - minimum), where=(maximum != minimum))
    return dataset

def get_blosum62(seq):
    blosum_list = []
    for i in seq:
        if i in blosum62:
            blosum_list.append(blosum62[i])
        else:
            blosum_list.append(blosum62['-'])
    blosum = np.array(blosum_list)
#     blosum = normalization(blosum)  # why did they get ride of normalization?
    feature = np.zeros((1002,20))
    idx = blosum.shape[0]
    feature[0:idx,:] = blosum
    return feature


def make_tensor(path):
    data = pd.read_csv(path)
    sequences = data['sequence'].values
    labels = data['label'].values
    evolution = torch.zeros(len(sequences),1002,20)
    lengths = []
    for i in range(len(sequences)):
        lengths.append((len(sequences[i])))
        temp = get_blosum62(sequences[i])
        evolution[i,:,:] = torch.Tensor(temp)

    return evolution, torch.Tensor(lengths), torch.Tensor(labels) 

In [3]:
class dvib(nn.Module):
    def __init__(self,k,out_channels, hidden_size):
        super(dvib, self).__init__()
        
        self.conv = torch.nn.Conv2d(in_channels=1,
                            out_channels = out_channels,
                            kernel_size = (1,20),
                            stride=(1,1),
                            padding=(0,0),
                            )
        
        self.rnn = torch.nn.GRU(input_size = out_channels,  
                                hidden_size = hidden_size,
                                num_layers = 2,
                                bidirectional = True,
                                batch_first = True,
                                dropout = 0.2
                              )
        
        self.fc1 = nn.Linear(hidden_size*4, hidden_size*4)
#         self.fc2 = nn.Linear(1024,1024)
        self.enc_mean = nn.Linear(hidden_size*4+578,k)
        self.enc_std = nn.Linear(hidden_size*4+578,k)
        self.dec = nn.Linear(k, 2)
        
        self.drop_layer = torch.nn.Dropout(0.5)
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.constant_(self.fc1.bias, 0.0)
#         nn.init.xavier_uniform_(self.fc2.weight)
#         nn.init.constant_(self.fc2.bias, 0.0)
        nn.init.xavier_uniform_(self.enc_mean.weight)
        nn.init.constant_(self.enc_mean.bias, 0.0)
        nn.init.xavier_uniform_(self.enc_std.weight)
        nn.init.constant_(self.enc_std.bias, 0.0)
        nn.init.xavier_uniform_(self.dec.weight)
        nn.init.constant_(self.dec.bias, 0.0)
   
        
        
    def cnn_gru(self,x,lens):
#         print(x.shape)
        x = x.unsqueeze(1)
#         print('after first unsqueeze: ', x.shape)
        x = self.conv(x)
#         print('after conv: ', x.shape)   
        x = torch.nn.ReLU()(x)
#         print('shape after relu: ', x.shape,type(x))
        x = x.squeeze(3)
#         print('shape after squeeze: ', x.shape)
#         x = x.view(x.size(0),-1)
        x = x.permute(0,2,1)
#         print('shape after permute: ', x.shape)
#         print(type(lens))
        gru_input = pack_padded_sequence(x,lens,batch_first=True)
        output, hidden = self.rnn(gru_input)
        print('hidden layer: ', hidden.shape)
        a = hidden[-1]
        b = hidden[-2]
        c = hidden[-3]
        d = hidden[-4]
        print(torch.equal(a, b)) # T
        print(torch.equal(a, c)) # T
        print(torch.equal(a, d)) # F
        print(torch.equal(b, c)) # T
        print(torch.equal(b, d)) # F
        print(torch.equal(c, d)) # T
        output_all = torch.cat([hidden[-1],hidden[-2],hidden[-3],hidden[-4]],dim=1)
#         print("output_all.shape:",output_all.shape)    
        return output_all
    
        
    def forward(self, pssm, lengths, FEGS): 
        cnn_vectors = self.cnn_gru(pssm, lengths)
        feature_vec = torch.cat([cnn_vectors, FEGS], dim = 1)
      
        enc_mean, enc_std = self.enc_mean(feature_vec), f.softplus(self.enc_std(feature_vec)-5)
        eps = torch.randn_like(enc_std)
        latent = enc_mean + enc_std*eps
        
        outputs = f.sigmoid(self.dec(latent))
#         print(outputs.shape)

        return outputs, enc_mean, enc_std, latent

In [4]:
# initialize model object
device = torch.device('cpu')
out_channels = 128
hidden_size = 512
beta = 2#0，1，3，4，5，6
k = 1024
model = dvib(k, out_channels, hidden_size).to(device)

In [5]:
class DBUnpickler(pickle.Unpickler):

    def __init__(self, file):
        super().__init__(file)

    def persistent_load(self, pid):
        # This method is invoked whenever a persistent ID is encountered.
        # Here, pid is the tuple returned by DBPickler.
        lib, className, file, device, unknown = pid
        return className()

In [6]:
with open('../OriginalModel/archive/data.pkl', 'rb') as file:
    pretrainedModel = DBUnpickler(file).load()

In [7]:
# initialize model weights with weights from ToxIBTL paper
for param, weights in zip(model.parameters(), pretrainedModel['model'].values()):
    param.data = weights.nan_to_num()

In [8]:
# retrieve FEGS embeddings for test set
test_fegs = []
for i in range(350):
    test_data = sio.loadmat('../../data/multiFEGS/test-chunks/' + str(i) + '.mat')['FV']
    test_fegs.append(test_data)

test_FEGS = torch.Tensor(normalization(np.vstack(test_fegs)))

In [9]:
# note: takes a hot second to run, try to run ONCE
test_path = './newData/test.csv'
test_pssm, test_len, test_label = make_tensor(test_path)

In [10]:
test_data = DataLoader(TensorDataset(test_pssm, test_len, test_FEGS, test_label), batch_size=100)

In [17]:
import numpy as np
np.set_printoptions(threshold=200)

In [19]:
print(test_fegs)

[array([[0.99512195, 0.99512195, 0.99512195, ..., 0.        , 0.        ,
        0.        ],
       [0.99315068, 0.99315068, 0.99315068, ..., 0.        , 0.        ,
        0.        ],
       [0.99516908, 0.99516908, 0.99516908, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.93333333, 0.93333333, 0.93333333, ..., 0.        , 0.        ,
        0.        ],
       [0.99516908, 0.99516908, 0.99516908, ..., 0.        , 0.        ,
        0.        ],
       [0.99771167, 0.99771167, 0.99771167, ..., 0.        , 0.        ,
        0.        ]]), array([[0.996     , 0.996     , 0.996     , ..., 0.        , 0.        ,
        0.        ],
       [0.99512195, 0.99512195, 0.99512195, ..., 0.        , 0.        ,
        0.        ],
       [0.99526066, 0.99526066, 0.99526066, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.99547511, 0.99547511, 0.99547511, ..., 0.        , 0.        ,
        0.        ],
       [0.99253731, 0.99253731, 0

In [11]:
# evaluate model
# note: takes a couple hours to run on a local computer
correct = 0
y_pre = []
y_test = []
with torch.no_grad():
    for batch_idx, (sequences, lengths, FEGS, labels) in enumerate(test_data):
        if (len(lengths) != 100):
            break
        print('batch ' + str(batch_idx+1) + ' of ' + str(len(test_data)))
        seq_lengths, perm_idx = lengths.sort(dim=0, descending=True)
        seq_tensor = sequences[perm_idx].to(device)
        FEGS_tensor = FEGS[perm_idx].to(device)
        label = labels[perm_idx].long().to(device)
        y_test.extend(label.cpu().detach().numpy())

        y_pred, end_means, enc_stds, latent = model(seq_tensor, seq_lengths, FEGS_tensor)
        y_pre.extend(y_pred.argmax(dim=1).cpu().detach().numpy())

        _, pred = torch.max(y_pred, 1) 

        correct += pred.eq(label).sum().item()

batch 1 of 350
hidden layer:  torch.Size([4, 100, 512])
True
True
True
True
True
True
batch 2 of 350


/Applications/PyMOL.app/Contents/envs/tox/lib/python3.9/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


hidden layer:  torch.Size([4, 100, 512])
True
True
True
True
True
True
batch 3 of 350
hidden layer:  torch.Size([4, 100, 512])
True
True
True
True
True
True
batch 4 of 350
hidden layer:  torch.Size([4, 100, 512])
True
True
True
True
True
True
batch 5 of 350
hidden layer:  torch.Size([4, 100, 512])
True
True
True
True
True
True
batch 6 of 350


KeyboardInterrupt: 

In [ ]:
# with open('./y_pre.txt', 'w') as y_pre_f:
#     for i in y_pre:
#         y_pre_f.write(str(i) + '\n')

In [ ]:
# with open('./y_test.txt', 'w') as y_test_f:
#     for i, label in enumerate(test_label):
#         if i < 41000:
#             y_test_f.write(str(int(label)) + '\n')

In [ ]:
# y_pre = []
# with open('./y_pre.txt') as file:
#     for line in file.readlines():
#         y_pre.append(int(line))

In [ ]:
# y_test = []
# with open('./y_test.txt') as file:
#     for line in file.readlines():
#         y_test.append(int(line))

In [ ]:
correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
for pred, label in zip(y_pre, y_test):
    if pred == label:
        correct += 1
    if label == 1 and pred == 1:
        tp += 1
    elif label == 1 and pred == 0:
        fn += 1
    elif label == 0 and pred == 0:
        tn += 1
    elif label == 0 and pred == 1:
        fp += 1

print('accuracy: ', correct / len(y_test))
print('ppv: ', tp / (tp + fp))
print('npv: ', tn / (tn + fn))
print('sensitivity: ', tp / (tp + fn))
print('specificity: ', tn / (tn + fp))